In [1]:
from load_data_json import LoadDataset 
from preprocess import VectorSpaceModel
import numpy as np
import time
from sklearn.decomposition import TruncatedSVD

## Load the dataset
Functions to load the dataset and the given queries with associated relevant documents.

In [2]:
dataset = LoadDataset("./data/med/med.json", "./data/med/queries.json", "./data/med/qrels-treceval.txt")

In [3]:
m,n = dataset.doc_matrix.shape
dataset.doc_matrix.nnz/(m*n)

0.004691013235037071

In [4]:
epsilon = np.finfo(np.float64).eps
sqrt_epsilon = np.sqrt(epsilon)
v = np.random.rand(n)
v = v / np.linalg.norm(v)

In [5]:
def preprocess(k, A, v):
    m, n = A.shape
    q = [np.zeros(n) for i in range(k)]
    beta = 0
    alpha = 0
    q[0] = v
    memo = {}
    for i in range(k-1):
        q_hat = A.dot(q[i])
        w = A.T.dot(q_hat) - beta * q[i-1]
        alpha = w.dot(q[i])
        w = w - alpha * q[i]
        for j in range(i):
            w_dotq = w @ q[j]
            w = w - w_dotq * q[j]
        beta = np.linalg.norm(w)
        if beta == 0:
            break
        q[i+1] = w / beta
    return q

In [6]:
q = preprocess(300, dataset.doc_matrix, v)

In [7]:
def response(q, query, A):
    k = len(q)
    s_hat = A.T.dot(query)
    m, n = A.shape
    s = np.zeros(n) 
    for i in range(k-1):
        q_dot_query = q[i] @ s_hat
        s = s + q_dot_query * q[i]

    return s



In [8]:
query = dataset.query_vectors[:,0]
similaries = response(q, query, dataset.doc_matrix)

In [9]:
similaries

array([ 0.00551104,  0.00471182,  0.00527557, ...,  0.00197232,
       -0.00063495, -0.00961434])

# Vector Space Model ranked retrieval

Create the `vsm()` object.

In [10]:
vsm = VectorSpaceModel(dataset.doc_matrix)
m, n = vsm.A.shape

In [11]:
vsm.preprocess(300)

In [12]:
epsilon = np.finfo(np.float64).eps
epsilon = np.sqrt(epsilon)
epsilon

1.4901161193847656e-08

In [13]:
count = 0
for i in range(300):
    x = vsm.lanczos_vectors[i] @ vsm.lanczos_vectors[i]
    print(x)

0.9999999999999999
1.0000000000000002
1.0
1.0000000000000002
1.0
0.9999999999999998
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0000000000000002
0.9999999999999999
1.0
1.0000000000000002
1.0
1.0
1.0
1.0000000000000002
1.0000000000000002
1.0
1.0
1.0
0.9999999999999998
0.9999999999999999
0.9999999999999997
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0000000000000004
1.0000000000000002
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0
1.0000000000000002
1.0
0.9999999999999999
1.0000000000000002
1.0
1.0000000000000002
1.0
0.9999999999999998
1.0
0.9999999999999998
1.0
1.0
1.0000000000000002
1.0000000000000002
0.9999999999999999
1.0
0.9999999999999999
1.0000000000000002
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
0.9999999999999998
1.0
1.0
0.9999999999999998
1.0
1.0000000000000004
0.9999999999999999
1.0
0.9999999999999998
1.0
0.9999999999999998
1.0
1.0
1.0
1.0000000000000002
0.9999999999999999
1.0
1.0
0.99999

In [14]:
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)

In [15]:
Ak = svd.fit_transform(vsm.A)

In [16]:
svd.components_.shape

(50, 1033)

In [17]:
dataset.query_vectors[:,0].T.dot(Ak).dot(np.diag(svd.singular_values_)).dot(svd.components_)

array([[0.00660732, 0.01935342, 0.00466614, ..., 0.00988035, 0.00471508,
        0.03267004]])

In [18]:
epsilon = np.finfo(np.float64).eps
epsilon

2.220446049250313e-16

In [19]:
vsm.A.shape

(13004, 1033)

In [20]:
vsm.response(dataset.query_vectors[:,0])

In [21]:
vsm.scores
scores = vsm.scores/vsm.norms
sorted_indices = np.argsort(-np.abs(scores))
sorted_indices[:37]

array([ 71, 211, 512, 499, 184,  14, 141, 167, 168, 510, 169, 180, 166,
       163, 183, 170, 137, 165, 511, 171, 210,  12, 185,  78, 505, 179,
       181, 508,  13, 498, 500, 503, 502, 509, 506, 504, 548], dtype=int64)

In [22]:
vsm.scores[212]

0.03507347278476517

In [23]:
sorted(sorted_indices[:37])

[12,
 13,
 14,
 71,
 78,
 137,
 141,
 163,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 179,
 180,
 181,
 183,
 184,
 185,
 210,
 211,
 498,
 499,
 500,
 502,
 503,
 504,
 505,
 506,
 508,
 509,
 510,
 511,
 512,
 548]

In [24]:
np.sort(x)

AxisError: axis -1 is out of bounds for array of dimension 0

In [ ]:
u, s, v, lsi_norm = vsm.lsi_preprocess(50)

In [ ]:
u

array([[-0.01029994, -0.00024417, -0.00393161, ..., -0.01098249,
         0.0126084 ,  0.01114007],
       [-0.00104717, -0.00155091, -0.00216838, ..., -0.00034772,
         0.00027169,  0.00053776],
       [ 0.00320095,  0.00047048,  0.00066028, ..., -0.00015159,
        -0.00011783,  0.00050536],
       ...,
       [-0.00530743, -0.00359512,  0.00454236, ...,  0.00017045,
        -0.00029722,  0.00067153],
       [ 0.00090966,  0.00035695, -0.00091288, ..., -0.00123334,
         0.00037175,  0.00064278],
       [-0.00056484,  0.00134659, -0.00144216, ..., -0.00695932,
         0.0039723 ,  0.00226147]])

In [ ]:
u = u[:,::-1]; s = s[::-1]; v = v[::-1,:]

In [ ]:
u.dot(np.diag(s)).dot(v) - vsm.A

matrix([[-3.87227049e-03, -9.77839057e-04, -7.85574710e-04, ...,
          1.07582933e-03,  1.15551230e-03,  2.42526126e-03],
        [-9.02233837e-04, -1.45434447e-03, -5.13185673e-04, ...,
         -2.75960154e-04,  1.69251330e-06,  3.67502786e-04],
        [-9.38994440e-05,  6.91317048e-05, -5.97949878e-04, ...,
         -1.15224932e-04,  4.47292738e-06,  1.18305983e-04],
        ...,
        [ 3.35008462e-04,  8.04557320e-04, -1.23284568e-04, ...,
         -5.98768755e-05,  1.39951350e-04,  8.92004405e-05],
        [ 1.55669149e-04, -6.79121558e-04, -4.75231695e-05, ...,
         -6.19752470e-05, -7.51170537e-05,  3.44085287e-05],
        [ 1.13989658e-03, -2.74357421e-04,  3.42133986e-04, ...,
          3.44649120e-04,  3.08890774e-04,  3.89212740e-05]])

In [ ]:
vsm.A

<13004x1033 sparse matrix of type '<class 'numpy.float64'>'
	with 63015 stored elements in Compressed Sparse Column format>

In [ ]:
vsm.scores

array([ 0.00227034, -0.00350398,  0.00197769, ...,  0.00165225,
        0.00061674,  0.00149766])

In [ ]:
res = vsm.lsi_response(u, s, v, dataset.query_vectors[:,0])
res 

array([[ 0.00239841, -0.00077874, -0.00145914, ..., -0.00012103,
        -0.00115709,  0.00735473]])

In [ ]:
epsilon = np.finfo(np.float64).eps
epsilon

2.220446049250313e-16

In [ ]:
scores = vsm.scores[::-1]


In [ ]:
np.linalg.norm(scores - res)

0.9658148905218698

In [ ]:
res/lsi_norm

array([[ 0.00418371, -0.00199911, -0.00483557, ..., -0.00043685,
        -0.00765655,  0.0217793 ]])

In [ ]:
vsm.scores/np.sqrt(vsm.norms)

TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable sqrt method

In [ ]:
np.sqrt(8.30)

2.8809720581775866